In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from catboost import CatBoostRegressor, Pool

# import torch
# import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_ace_24 = pd.read_csv("Ace_погружение_24часа.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_24 = pd.read_csv("Discover_погружение_24часа_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_ace_autocorr = pd.read_csv("Ace_погружение_автокорр.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_autocorr = pd.read_csv("Discover_погружение_автокорр_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])

In [3]:
datasets = [data_ace_24, data_discover_24, data_ace_autocorr, data_discover_autocorr]

In [4]:
for dataset in datasets:
    dataset.set_index('datetime', inplace=True)
    dataset.dropna(inplace=True)
    if not dataset.index.is_monotonic_increasing:
        dataset.sort_index(inplace=True)

In [5]:
from_index_dst_data = data_ace_24.columns.get_loc('Dst') #индекс, с которого начинаются данные о нынешнем и предыдущих значениях Dst, величине ММП и параметрах СВ
to_index_of_dst_data_24 = 24+1
to_index_of_dst_data_a = 43+1
hour_horizont = 3
target_columns = ['Dst +1', 'Dst +2', 'Dst +3']

### Доменная адаптация 1 в 1 - линейная регрессия

In [6]:
def adaptation_one_to_one(data_disc, data_ace, shifts='24'):
    indexes_for_shifts = {'24': to_index_of_dst_data_24, 'a': to_index_of_dst_data_a}
    index = indexes_for_shifts[shifts]
    
    common_index = data_ace.index.unique().intersection(data_disc.index.unique()) # нахожу общие индексы
    data_disc = data_disc.loc[common_index]
    data_ace = data_ace.loc[common_index]

    # при разбиении в Y лежит весь массив с дст (погруженный), при обучении нужно убрать лишнее
    # в Х лежат данные без значений дст, при обучении добавить
    # это нужно для того, чтобы можно было не задавать горизонт предсказания при адаптации при каждом запуске функции
    x_train_ace, x_test_ace, y_train_ace, y_test_ace = \
    train_test_split(data_ace.iloc[:, from_index_dst_data:], data_ace.iloc[:, :from_index_dst_data], test_size=0.15, shuffle=False)

    x_train_disc, x_test_disc, y_train_disc, y_test_disc = \
    train_test_split(data_disc.iloc[:, from_index_dst_data:], data_disc.iloc[:, :from_index_dst_data], test_size=0.15, shuffle=False)

    model = LinearRegression()
    x_pred = pd.DataFrame()
    
    names_of_adaptated_columns = x_test_disc.iloc[:, index:].columns
    indexes_of_adaptated_columns = x_test_disc.iloc[:, index:].index

    dst_data_test = x_test_ace.iloc[:, :index]
    dst_data_train = x_train_ace.iloc[:, :index]
    
    for col in x_train_ace.iloc[:, index:].columns:
        model.fit(x_train_disc[col].to_frame(), x_train_ace[col].to_frame())
        x_pred_column = model.predict(x_test_disc[col].to_frame())
        x_pred_column = x_pred_column.flatten()
        x_pred[col] = x_pred_column

    x_pred.columns = names_of_adaptated_columns
    x_pred.index = indexes_of_adaptated_columns
    
    x_pred = pd.merge(dst_data_test, x_pred, left_index=True, right_index=True, how='inner')
    
    return x_train_disc, x_pred, y_train_disc, y_test_disc

In [7]:
x_train_disc_24, x_pred_24, y_train_disc_24, y_test_disc_24 = adaptation_one_to_one(data_discover_24, data_ace_24, shifts='24')

In [8]:
x_train_disc_a, x_pred_a, y_train_disc_a, y_test_disc_a = adaptation_one_to_one(data_discover_autocorr, data_ace_autocorr, shifts='a')

### Модели

In [9]:
model_cat = CatBoostRegressor(iterations=500,
                              depth=2,
                              learning_rate=1,
                              loss_function='RMSE',
                              logging_level='Silent',
                              early_stopping_rounds=40)

pipe = make_pipeline(StandardScaler(), LinearRegression())

In [10]:
def linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for i in range(len(y_trains)):
        pipe.fit(x_train, y_trains[i])
        y_pred = pipe.predict(x_test)

        r_2 = metrics.r2_score(y_tests[i], y_pred)
        mse = metrics.mean_squared_error(y_tests[i], y_pred)

        # print(f'Прогноз на {i+1} час:')
        # print(f'MSE={mse} и r^2={r_2} линейная регрессия, погружение - {depth}')

        new_row = {
            'Способ обучения': type_train_test,
            'Глубина погружения': depth,
            'Модель': 'Линейная регрессия',
            'Горизонт предсказания': f'Dst +{i+1}',
            'R_2 score': r_2,
            'MSE': mse
        }
        results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
    return results

In [11]:
def linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, lambda_values, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for lambda_val in lambda_values:
        for i in range(len(y_trains)):
            ridge_reg = Ridge(lambda_val)
            ridge_reg.fit(x_train, y_trains[i])
            y_pred = ridge_reg.predict(x_test)
            
            r_2 = metrics.r2_score(y_tests[i], y_pred)
            mse = metrics.mean_squared_error(y_tests[i], y_pred)
            
            # print(f'Прогноз на {i+1} час:')
            # print(f'MSE={mse} и r^2={r_2} с Lambda={lambda_val} линейная регрессия с Ridge-регуляризацией, погружение - {depth}')

            new_row = {
                'Способ обучения': type_train_test,
                'Глубина погружения': depth,
                'Модель': f'Линейная регрессия c регуляризацией lambda={lambda_val}',
                'Горизонт предсказания': f'Dst +{i+1}',
                'R_2 score': r_2,
                'MSE': mse
            }

            results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
    return results

In [12]:
def catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for i in range(len(y_trains)):
        train_pool = Pool(x_train, y_trains[i], cat_features=[])
        test_pool = Pool(x_test, y_tests[i], cat_features=[])

        model_cat.fit(train_pool)
        y_pred = model_cat.predict(test_pool)

        r_2 = metrics.r2_score(y_tests[i], y_pred)
        mse = metrics.mean_squared_error(y_tests[i], y_pred)

        # print(f'Прогноз на {i+1} час:')
        # print(f'MSE={mse} и r^2={r_2}  CatBoost, погружение - {depth}')

        new_row = {
            'Способ обучения': type_train_test,
            'Глубина погружения': depth,
            'Модель': 'CatBoost',
            'Горизонт предсказания': f'Dst +{i+1}',
            'R_2 score': r_2,
            'MSE': mse
        }

        results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)  
    return results

#### Создание пустого датафрейма для записи результатов

In [13]:
results = pd.DataFrame()
columns = ['Способ обучения', 'Глубина погружения', 'Модель', 'Горизонт предсказания', 'R_2 score', 'MSE']

results = pd.DataFrame(columns=columns)

### Обучение на данных ACE, тестирование на адаптированных данных DSCVR

In [14]:
type_train_test = 'ACE-adaptated_DSCVR'

#### Погружение - 24 часа по всем переменным

In [15]:
x_train, x_test = x_train_disc_24, x_pred_24
ys_train, ys_test = [], []

for i in range(2, -1, -1):
    ys_train.append(y_train_disc_24[target_columns[i]])
    ys_test.append(y_test_disc_24[target_columns[i]])

In [16]:
y_train_3, y_train_2, y_train_1 = ys_train
y_test_3, y_test_2, y_test_1 = ys_test

In [17]:
lambda_values = [1000000, 5000000, 10000000]

In [18]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)

#### Погружение с учётом автокорреляционной функции

In [19]:
x_train, x_test = x_train_disc_a, x_pred_a
ys_train, ys_test = [], []

for i in range(2, -1, -1):
    ys_train.append(y_train_disc_a[target_columns[i]])
    ys_test.append(y_test_disc_a[target_columns[i]])

In [20]:
y_train_3, y_train_2, y_train_1 = ys_train
y_test_3, y_test_2, y_test_1 = ys_test

In [21]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)

In [22]:
results

,Способ обучения,Глубина погружения,Модель,Горизонт предсказания,R_2 score,MSE
0,ACE-adaptated_DSCVR,24 часа,Линейная регрессия,Dst +1,0.945977,8.804227
1,ACE-adaptated_DSCVR,24 часа,Линейная регрессия,Dst +2,0.865061,21.989349
2,ACE-adaptated_DSCVR,24 часа,Линейная регрессия,Dst +3,0.786739,34.747432
3,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +1,0.894698,17.161222
4,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +2,0.809512,31.041581
5,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +3,0.738979,42.529180
6,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=500...,Dst +1,0.820688,29.222838
7,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=500...,Dst +2,0.751049,40.568605
8,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=500...,Dst +3,0.694620,49.756675
9,ACE-adaptated_DSCVR,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +1,0.772045,37.150198


In [23]:
results.to_excel('результаты обучения - адаптация линрег.xlsx', index=False)

In [24]:
results.to_csv('результаты обучения - адаптация линрег.csv', sep=';', encoding='utf-8')